In [11]:
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import display, Markdown, SVG, HTML
import pandas as pd
import altair as alt
import re
import pickle
from utils import seconds_to_ms, ms_to_seconds
from benchmark_result import get_benchmark_results, BenchmarkType, SchedulerType, get_broadcast_source, get_async_source
from benchmarks.utils import echo
from benchmarks.throughput import make_benchmark, make_multiple_message_benchmark

In [13]:
#benchmark_results = get_benchmark_results()
from benchmark_result import BenchmarkResult, Result 
with open('saved_results.pkl', 'rb') as saved_results:
    benchmark_results = pickle.load(saved_results)

## ipyparallel benchmark results ##

# time_broadcast

This benchmark comes from benchmarking the runtime of sending and returning different size arrays to different number of engines with the echo function on various scheduler implementations


In [4]:
??echo

Signature: echo(delay=0)
Docstring: <no docstring>
Source:   
def echo(delay=0):
    def inner_echo(x, **kwargs):
        import time

        if delay:
            time.sleep(delay)
        return x

    return inner_echo
File:      ~/ipyparallel_master_project/benchmarks/utils.py
Type:      function


In [5]:
??make_benchmark

Signature: make_benchmark(benchmark_name, get_view)
Docstring: <no docstring>
Source:   
def make_benchmark(benchmark_name, get_view):
    class ThroughputSuite:
        param_names = ['Number of engines', 'Number of bytes']
        timer = timeit.default_timer
        timeout = 120
        params = [engines, byte_param]

        view = None
        client = None
        reply = None

        def setup(self, number_of_engines, number_of_bytes):
            self.client = ipp.Client(profile='asv')
            self.view = get_view(self)
            self.view.targets = list(range(number_of_engines))
            wait_for(lambda: len(self.client) >= number_of_engines)

        def time_broadcast(self, engines, number_of_bytes):
            self.reply = self.view.apply_sync(
                echo, np.array([0] * number_of_bytes, dtype=np.int8)
            )

        def teardown(self, *args):
            if self.client:
                self.client.close()

    return ThroughputSuite
File:     

### Results from running benchmark on 64 cores


In [14]:
source = get_broadcast_source(benchmark_results)

In [15]:
dview = pd.DataFrame(source['DirectView']) 
dview['Scheduler name'] = 'DirectView'
dview['Speedup'] = 1

In [16]:
datas = []
for scheduler_name, scheduler_results in source.items():
    data = pd.DataFrame(scheduler_results) 
    data['Scheduler name'] = scheduler_name
    data['Speedup'] = dview['Duration in ms'] / data['Duration in ms']
    datas.append(data)
data = pd.concat(datas)
data

,Duration in ms,Number of bytes,Number of engines,Scheduler name,Speedup
0,9.45,1000,1,NonCoalescing,0.484656
1,9.47,10000,1,NonCoalescing,0.587117
2,17.83,100000,1,NonCoalescing,0.722378
3,89.25,1000000,1,NonCoalescing,0.873501
4,172.51,2000000,1,NonCoalescing,0.858733
...,...,...,...,...,...
35,677.21,1000,1024,DirectView,1.000000
36,661.01,10000,1024,DirectView,1.000000
37,760.37,100000,1024,DirectView,1.000000
38,1979.30,1000000,1024,DirectView,1.000000


In [17]:
ldata = data[data['Number of bytes'] == 2000_000]
alt.Chart(ldata).mark_line(point=True).encode(
    alt.X(
        'Number of engines',
        scale=alt.Scale(type='log', base=2)
    ),
    alt.Y(
        'Duration in ms',
        scale=alt.Scale(type='log')
    ),
    color='Scheduler name:N',
    tooltip='Duration in ms',
).configure_axis(labelFontSize=20, titleFontSize=20).properties(title='Runtime of apply using DirectView', width=1080).interactive().display(renderer='svg')

alt.Chart(...)

In [32]:
for scheduler_name in data['Scheduler name'].unique():
    scheduler_data = data[data['Scheduler name'] == scheduler_name]
    alt.Chart(scheduler_data).mark_line(point=True).encode(
        alt.X(
            'Number of bytes',
            scale=alt.Scale(type='log')
        ),
        alt.Y(
            'Duration in ms',
            scale=alt.Scale(type='log')

        ),
        color='Number of engines:N',
        tooltip='Duration in ms',
    
    ).properties(title=scheduler_name, width=800).interactive().display(renderer='svg')


alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

In [33]:
display(Markdown(f'## Results for duration[DirectView]/duration[scheduler]'))
for scheduler_name in data['Scheduler name'].unique():
    if scheduler_name == 'DirectView':
        continue
    alt.Chart(data[data['Scheduler name'] == scheduler_name]).mark_line(point=True).encode(
        alt.X(
            'Number of bytes',
            scale=alt.Scale(type='log')
        ),
        alt.Y(
             'Speedup',
        ),
        color='Number of engines:N',
        tooltip='Number of engines',
    
    ).properties(title=scheduler_name, width=800).interactive().display(renderer='svg')

## Results for duration[DirectView]/duration[scheduler]

alt.Chart(...)

alt.Chart(...)

In [34]:

for engine in data['Number of engines'].unique():
    display(Markdown(f'### Running on {engine} engines'))
    alt.Chart(data[(data['Number of engines'] == engine) & (data['Scheduler name'] != 'DirectView')]).mark_line(point=True).encode(
        alt.X(
            'Number of bytes',
            scale=alt.Scale(type='log')
        ),
        alt.Y(
            'Speedup',
        ),
        color='Scheduler name:N',
        tooltip='Duration in ms',
    ).properties(title='schedulers vs directView', width=800).interactive().display(renderer='svg')

### Running on 1 engines

alt.Chart(...)

### Running on 2 engines

alt.Chart(...)

### Running on 16 engines

...

alt.Chart(...)

### Running on 512 engines

alt.Chart(...)

### Running on 1024 engines

alt.Chart(...)

In [35]:
data['combined']=  data['Scheduler name'] + ' ' + data['Number of bytes'].astype(str)
alt.Chart(data[data['Scheduler name'] != 'DirectView']).mark_line(point=True).encode(
    alt.X(
        'Number of engines',
            scale=alt.Scale(type='log')
    ),
    alt.Y(
        'Speedup',
    ),
    color='Number of bytes:N',
    strokeDash=alt.StrokeDash(shorthand='Scheduler name', legend=None),
    tooltip='combined',

).properties(title='schedulers vs directView scaling engines', width=800).interactive().display(renderer='svg')


alt.Chart(...)

In [18]:
lol = data[data['Number of engines'] == 1024]
lol

,Duration in ms,Number of bytes,Number of engines,Scheduler name,Speedup
35,617.38,1000,1024,NonCoalescing,1.096910
36,619.59,10000,1024,NonCoalescing,1.066851
37,652.45,100000,1024,NonCoalescing,1.165407
38,1132.67,1000000,1024,NonCoalescing,1.747464
39,4143.30,2000000,1024,NonCoalescing,0.951425
35,176.21,1000,1024,Coalescing,3.843198
36,221.57,10000,1024,Coalescing,2.983301
37,354.27,100000,1024,Coalescing,2.146301
38,1699.83,1000000,1024,Coalescing,1.164411
39,4317.67,2000000,1024,Coalescing,0.913002
